In [9]:
import joblib
import pandas as pd
# Load the saved model and scaler
rf_model = joblib.load('rf_model.pkl')
scaler = joblib.load('scaler.pkl')
print("Model and scaler loaded successfully.")


Model and scaler loaded successfully.


In [10]:
# Feature names from your feature importance
feature_names = [
    'heartRate_num', 'spo2_num', 'temperature_num', 'systolicBP', 'diastolicBP',
    'age', 'stepsTaken', 'calorieIntake', 'sleepHours', 'waterIntakeMl',
    'mealsSkipped', 'exerciseMinutes', 'bathroomVisits'
]


In [11]:
# Function to get user input
def get_user_input():
    print("Enter the following feature values:")
    user_data = {}
    for feature in feature_names:
        while True:
            try:
                value = float(input(f"Enter {feature}: "))
                # Basic range validation
                if feature == 'spo2_num' and (value < 0 or value > 100):
                    print(f"{feature} must be between 0 and 100.")
                    continue
                if feature in ['heartRate_num', 'systolicBP', 'diastolicBP', 'age', 'stepsTaken', 'calorieIntake', 'sleepHours', 'waterIntakeMl', 'mealsSkipped', 'exerciseMinutes', 'bathroomVisits'] and value < 0:
                    print(f"{feature} cannot be negative.")
                    continue
                if feature == 'temperature_num' and (value < 30 or value > 45):
                    print(f"{feature} must be between 30 and 45°C.")
                    continue
                user_data[feature] = value
                break
            except ValueError:
                print(f"Invalid input for {feature}. Please enter a numeric value.")
    return user_data

In [12]:
# Get user input
user_data = get_user_input()

# Convert to DataFrame
input_df = pd.DataFrame([user_data], columns=feature_names)

# Scale the input
input_scaled = scaler.transform(input_df)

# Predict and get probabilities
prediction = rf_model.predict(input_scaled)[0]
prob = rf_model.predict_proba(input_scaled)[0]

# Map prediction to class label
class_label = "Stable (0)" if prediction == 0 else "At Risk (1)"

# Output results
print(f"\nPrediction: The person is {class_label}.")
print(f"Probability of Stable (0): {prob[0]:.2f}, At Risk (1): {prob[1]:.2f}")

# Address potential misprediction (Stable when expecting At Risk)
if prediction == 0:
    print("\nNote: Model predicted 'Stable'. If you expected 'At Risk', trying adjusted threshold (0.4):")
    prediction_adj = 1 if prob[1] > 0.4 else 0
    class_label_adj = "At Risk (1)" if prediction_adj == 1 else "Stable (0)"
    print(f"Adjusted Prediction: The person is {class_label_adj}.")

Enter the following feature values:


Enter heartRate_num:  90
Enter spo2_num:  99
Enter temperature_num:  35
Enter systolicBP:  100
Enter diastolicBP:  66
Enter age:  22
Enter stepsTaken:  100
Enter calorieIntake:  100
Enter sleepHours:  8
Enter waterIntakeMl:  1000
Enter mealsSkipped:  3
Enter exerciseMinutes:  10
Enter bathroomVisits:  3



Prediction: The person is At Risk (1).
Probability of Stable (0): 0.10, At Risk (1): 0.90
